Segmenting and Clustering Neighborhoodsoronto

ALL 3 tasks in 1 notebook

In [ ]:
Starting with import and preparations of env

In [ ]:
#!conda install -c conda-forge folium --yes
#!pip uninstall -y numpy
#!pip install numpy
#!pip install numpy==1.18
#!pip install scipy==1.1.0
#!pip install scikit-learn==0.21.3
#!pip install beautifulsoup4
#!pip install lxml
import requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML
from IPython.display import display_html
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

TASK 1 getting and preparing the data

In [ ]:
Read wikipedia page and create DF

In [ ]:
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(html_data.text, 'html.parser')
table_contents = []
table = soup.find('table')
u = 0
for row in table.findAll('td'):
    cell = {}
    u = u+1
    if 'Not assigned' in row.span.text:
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (
            ((((row.span.text).split('(')[1]).strip(')')).replace(' /', ',')).replace(')', ' ')).strip(' ')
        table_contents.append(cell)


df = pd.DataFrame(table_contents)
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto Stn A',
                                       'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto Business',
                                       'EtobicokeNorthwest': 'Etobicoke Northwest',
                                       'East YorkEast Toronto': 'East York/East Toronto',
                                       'MississaugaCanada Post Gateway Processing Centre': 'Mississauga'})
# replace the / value with ,
df.Neighborhood=df.Neighborhood.str.replace((" /"),(","))
print('size of dataframe is ', df.size)
print('shape  of dataframe is ', df.shape)
print('dimension of dataframe is ', df.ndim)

Task 2 - get locations

Get the coordinations for postal codes from csv and join it with my primary DF

In [ ]:
csvCoordPostalCodes = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
df_pc_coor = pd.read_csv(csvCoordPostalCodes)
df_joined = df.join(df_pc_coor.set_index('Postal Code'), on='PostalCode')

LEt's get only the Toronto data

In [ ]:
f_Toronto = df_joined[df_joined['Borough'].str.contains('Toronto',regex=False)]
print('size of dataframe is ', df_Toronto.size)
print('shape  of dataframe is ', df_Toronto.shape)
print('dimension of dataframe is ', df_Toronto.ndim)

In [ ]:

LEt's find coordinates for Toronto, Canada and prepare the map: